In [1]:
from postprocessing import *
import numpy as np
import time, os, random, shutil, pickle, ray
from cgp import *
import matplotlib.pyplot as plt
from configuration import config
import warnings
warnings.filterwarnings('ignore')

c:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [11]:
def rollout(dataset, policy):
    x, y, z = dataset
    count = 0
    for xi,yi,zi in zip(x,y,z):
        # input = [d[0], d[1]]
        # label = d[2]
        input = [xi, yi]
        output = policy.eval(*input)
        if type(output)!=float:
            continue
        # print(zi, output, zi == output)
        # if zi == output:
        #     count += 1
        count += np.abs(zi-output)
    return count / len(x)

def rollout2(dataset, policy):
    x, y = dataset
    z = [xx>=1.1*yy for xx,yy in zip(x,y)]
    count = 0
    for xi, yi, zi in zip(x,y,z):
        input = [xi, yi]
        output = policy.eval(*input)
        if output == zi:
            count += 1
    return count / 1000

def rollout3(dataset, policy):
    train, label = dataset
    count = 0
    for xy, l in zip(train, label):
        output = policy.eval(*xy)
        
        if not type(output)==np.ndarray:
            # print(output, l)
            if output == l:
                count += 1
    return count / 100

In [12]:
def get_data():
    num = 100
    dim = 10
    x, y, z = [], [], []
    for i in range(num):
        xi = np.random.rand(dim,)
        x.append(xi)
        yi = xi * (np.random.uniform([-0.15]*dim, [0.15]*dim) + 1)
        y.append(yi)
        
        zi = np.sum(np.sign(np.maximum((yi-xi)/xi-0.1, 0))*xi)
        z.append(zi)
        
    return [x, y, z]

def get_data2():
    num = 1000
    train, label = [], []
    for i in range(num):
        x = np.random.uniform(-100,100)
        y = x * (1+np.random.uniform(-0.2, 0.2))
        train.append(x)
        label.append(y)
    return [train, label]

def get_data3():
    num = 100
    train, label = [], []
    for i in range(num):
        x = np.random.randint([-20]*10, [20]*10)
        y = np.random.randint(-20, 20)
        train.append((x,y))
        label.append(x[x>=y].sum())
    return [train, label]

In [13]:
# 涨跌判断器
# fs = [
#         # Function(op.add, 2), 
#         Function(op.sub, 2), 
#         # Function(op.mul, 2), 
#         # Function(protected_div, 2),
#         Function(op.abs, 1),
#         Function(sign, 1),
#         Function(tenth, 1),
#     ]
fs = [
    Function(op.ge, 2, '≥'),
    # Function(Sub, 2, name='—'),
    # Function(Sign, 1, name='sgn'),
    Function(Mul, 2, name='x'),
    Function(Sum, 1, name='∑')
]
def Abs(x): return np.abs(x)
new_functions = [
    # Function(Add, 2, name='Add'),
    Function(Sub, 2, name='Sub'),
    # Function(Mul, 2, name='Mul'),
    # Function(Div, 2, name='Div'),
    # Function(Neg, 1, name='Neg'),
    Function(Abs, 1, name='Abs'),
    # Function(Max1, 1, name='Max2'),
    # Function(Sum, 1, name='Sum'),
    # Function(Min1, 1, name='Min'),
    Function(Sign, 1, name='Sgn'),
    Function(tenth, 1,)
    # Function(const_01, 0, name='0.1')
]

In [19]:
np.random.seed(123)
random.seed(123)
pop = create_population(config.MU+config.LAMBDA, 
                            input_dim=2, 
                            out_dim=1,
                            fs=fs,
                            # fs=new_functions,
                            n_cols=5,
                            level_back=5,
                            out_random_active=False)
# data = get_data()
# data = get_data2()
data = get_data3()

In [20]:
for g in range(10000):
    tick = time.time()
    fit_list = [rollout3(data, p) for p in pop]
    # fitness = ray.get(fit_list)
    # 优化方向是最大化
    for f, p in zip(fit_list, pop):
        p.fitness = f
    pop = evolve(pop, 0.15, config.MU, config.LAMBDA)
    print(g, 'time:', round(time.time()-tick, 3),'best fitness:', pop[0].fitness)
    if pop[0].fitness == 1.0:
        break

0 time: 0.169 best fitness: 0.1
1 time: 0.105 best fitness: 0.1
2 time: 0.071 best fitness: 0.1
3 time: 0.123 best fitness: 0.1
4 time: 0.129 best fitness: 0.1
5 time: 0.124 best fitness: 0.1
6 time: 0.075 best fitness: 0.1
7 time: 0.059 best fitness: 0.1
8 time: 0.039 best fitness: 0.1
9 time: 0.052 best fitness: 0.1
10 time: 0.049 best fitness: 0.1
11 time: 0.069 best fitness: 0.1
12 time: 0.062 best fitness: 0.1
13 time: 0.082 best fitness: 0.1
14 time: 0.077 best fitness: 0.1
15 time: 0.064 best fitness: 0.1
16 time: 0.066 best fitness: 0.1
17 time: 0.07 best fitness: 0.1
18 time: 0.065 best fitness: 0.1
19 time: 0.078 best fitness: 0.1
20 time: 0.069 best fitness: 0.1
21 time: 0.063 best fitness: 0.1
22 time: 0.053 best fitness: 0.1
23 time: 0.044 best fitness: 0.1
24 time: 0.063 best fitness: 0.1
25 time: 0.07 best fitness: 0.1
26 time: 0.059 best fitness: 0.11
27 time: 0.094 best fitness: 0.11
28 time: 0.1 best fitness: 0.13
29 time: 0.115 best fitness: 1.0


In [21]:
g = extract_computational_subgraph(pop[0])
visualize(g, to_file='test.png',input_names=['x','y'])

Operator notation of '≥'' is not available. The node id is shown instead.
Operator notation of 'x'' is not available. The node id is shown instead.
Operator notation of '∑'' is not available. The node id is shown instead.
Operator notation of '∑'' is not available. The node id is shown instead.


In [5]:
class Individual:
    def __init__(self, input_dim, out_dim, function_set, n_cols, level_back, out_random_active=False):
        self.n_cols = n_cols # number of cols (nodes) in a single-row CGP
        self.level_back = level_back # 后面的节点可以最远连接的前面节点的相对位置
        self.function_set = function_set
        self.max_arity = max(f.arity for f in fs)
        self.n_inputs = input_dim
        self.n_outputs = out_dim # 输出维度
        self.weight_range = [-1, 1]
        
        # 创建nodes
        self.nodes = []
        for pos in range(self.n_cols):
            self.nodes.append(self._create_random_node(pos))
        
        # 将最后n_outputs个node设为输出节点
        for i in range(1, self.n_outputs + 1):
            if out_random_active: # 输出节点有一定概率被关闭
                if random.random() < 0.8: self.nodes[-i].active = True
                else: self.nodes[-i].active = False
            else: self.nodes[-i].active = True
        
        self.fitness = None
        self._active_determined = False

    def _create_random_node(self, pos):
        node = Node(self.max_arity)
        node.i_func = random.randint(0, len(self.function_set) - 1)
        for i in range(self.function_set[node.i_func].arity):
            # 随机确定node的每个输入端口连接的是前面节点(column)的输出
            # node.i_inputs[i]记录前端父节点的idx
            node.i_inputs[i] = random.randint(max(pos - self.level_back, -self.n_inputs), pos - 1)
            node.weights[i] = 1.0 # random.uniform(self.weight_range[0], self.weight_range[1])
        node.i_output = pos
        return node

    def _determine_active_nodes(self):
        """
        Determine which nodes in the CGP graph are active
        """
        n_active = 0
        for node in reversed(self.nodes):
            if node.active:
                n_active += 1
                for i in range(self.function_set[node.i_func].arity):
                    i_input = node.i_inputs[i]
                    if i_input >= 0: self.nodes[i_input].active = True
        self.n_active = n_active
        if config.Verbose: print("# active genes: ", n_active)

    def eval(self, *args):
        """
        Given inputs, evaluate the output of this CGP individual.
        :return the final output value
        """
        if not self._active_determined:
            self._determine_active_nodes()
            self._active_determined = True
        print('aaa',args)
        # forward pass: evaluate
        for node in self.nodes:
            if node.active:
                inputs = []
                for i in range(self.function_set[node.i_func].arity): # 依次获得该node各维输入
                    i_input = node.i_inputs[i] # node的父节点idx
                    w = node.weights[i]
                    if i_input < 0:
                        inputs.append(args[-i_input - 1] * w)
                    else:
                        inputs.append(self.nodes[i_input].output * w)
                print('bbb', '\n',
                        self.function_set[node.i_func].name, '\n',
                        node.i_inputs,  '\n',
                        inputs)
                node.output = self.function_set[node.i_func](*inputs) # 执行计算
        if self.n_outputs == 1:
            return self.nodes[-1].output
        out = []
        for i in reversed(range(1, self.n_outputs + 1)):
            out.append(self.nodes[-i].output)
        return out

    def mutate(self, mut_rate=0.01):
        child = copy.deepcopy(self)
        for pos, node in enumerate(child.nodes):
            if random.random() < mut_rate:
                node.i_func = random.choice(range(len(self.function_set)))
            arity = self.function_set[node.i_func].arity
            for i in range(arity):
                if node.i_inputs[i] is None or random.random() < mut_rate:  # if the mutated function requires more arguments, then the last ones are None 
                    node.i_inputs[i] = random.randint(max(pos - self.level_back, -self.n_inputs), pos - 1)
                if node.weights[i] is None or random.random() < mut_rate:
                    node.weights[i] = 1.0
            node.active = False
        for i in range(1, self.n_outputs + 1):
            child.nodes[-i].active = True
        child.fitness = None
        child._active_determined = False
        return child

In [8]:
ind = Individual(2,1,fs,3,3,)
ind.nodes[0].i_func = 0
ind.nodes[0].i_inputs = [-1, -2]
ind.nodes[1].i_func = 1
ind.nodes[1].i_inputs = [-1, 0]
ind.nodes[2].i_func = 2
ind.nodes[2].i_inputs = [1, None]
for node in ind.nodes:
    print(fs[node.i_func].name, node.i_inputs)
data = get_data3()
train, label = data
count = 0
for xy, l in zip(train, label):
    output = ind.eval(*xy)
    print('ccc', output, l)
    if not type(output)==np.ndarray:
        # print(output, l)
        if output == l:
            count += 1
print(count / 100)

≥ [-1, -2]
x [-1, 0]
∑ [1, None]
aaa (array([-12,   5,   4,  -2,  16,  17, -14,  -5, -15,  19]), -3)
bbb 
 ≥ 
 [-1, -2] 
 [array([-12.,   5.,   4.,  -2.,  16.,  17., -14.,  -5., -15.,  19.]), -3.0]


TypeError: unsupported operand type(s) for *: 'bool' and 'NoneType'